### **3-1. Hyperclova X로 추출한 객체 및 관계 정보를 preprocessing.**

- results/extracted_result_example.txt의 data를 preprocessing.

- 출력 형식을 활용하여 graphDB의 node와 edge가 될 정보들을 dictionary로 저장.

In [1]:
nodes = {}
edges = []
line_list = []

with open("results/extracted_result_example.txt", 'r') as f:
    for line in f.readlines():
        if line != "\n":
            line_list.append(line)
        else:
            collected_node_lines = []
            collected_edge_lines = []
            
            for collected_line in line_list:
                if collected_line[0] != "(" and collected_line[-2] != ")":
                    continue
                if collected_line[1:3] == "객체":
                    collected_node_lines.append(collected_line)
                elif collected_line[1:3] == "관계":
                    collected_edge_lines.append(collected_line)
                
            temp_nodes = {}
            
            for node_line in collected_node_lines:
                node_line = node_line.strip()
                elements = node_line[1:-1].split("[구분]")
                if len(elements) != 4:
                    continue
                name = elements[1].strip()
                type = elements[2].strip()
                if type not in ["기업", "인물", "기술", "사건", "국가"]:
                    continue
                if name not in temp_nodes:
                    temp_nodes[name] = {'type':type}
                    
            name_set = set()
            
            for edge_line in collected_edge_lines:
                edge_line = edge_line.strip()
                elements = edge_line[1:-1].split("[구분]")
                if len(elements) != 7:
                        continue
                name_1 = elements[1].strip()
                name_2 = elements[2].strip()
                description = elements[3].strip()
                score = elements[4].strip()
                time = elements[5].strip()
                url = elements[6].strip()
                if name_1 in temp_nodes and name_2 in temp_nodes:
                    if temp_nodes[name_1]['type'] == "기업" and temp_nodes[name_2]['type'] == "기업":
                        edges.append({'source':name_1, 'dest':name_2, 'description': description, 'score':score, 'time':time, 'url':url})
                        edges.append({'source':name_2, 'dest':name_1, 'description': description, 'score':score, 'time':time, 'url':url})
                        name_set.add(name_1)
                        name_set.add(name_2)
                    elif temp_nodes[name_1]['type'] != "기업":
                        if temp_nodes[name_2]['type'] != "기업":
                            continue
                        if temp_nodes[name_2]['type'] == "기업":
                            edges.append({'source':name_1, 'dest':name_2, 'description': description, 'score':score, 'time':time, 'url':url})
                            name_set.add(name_1)
                            name_set.add(name_2)
                    elif temp_nodes[name_2]['type'] != "기업":
                        if temp_nodes[name_1]['type'] != "기업":
                            continue
                        if temp_nodes[name_1]['type'] == "기업":
                            edges.append({'source':name_2, 'dest':name_1, 'description': description, 'score':score, 'time':time, 'url':url})   
                            name_set.add(name_1)
                            name_set.add(name_2)  
            
            for node_line in collected_node_lines:
                node_line = node_line.strip()
                elements = node_line[1:-1].split("[구분]")
                if len(elements) != 4:
                    continue
                name = elements[1].strip()
                if name not in name_set:
                    continue
                type = elements[2].strip()
                if type not in ["기업", "인물", "기술", "사건", "국가"]:
                    continue
                description = elements[3].strip()
                if name not in nodes:
                    nodes[name] = {'type':type, 'description':description}
                            
            line_list = []

### **3-2. node 및 edge 정보들을 csv 파일로 저장.**

- node는 type 별(기업: results/company.csv, 인물: results/people.csv, 기술 : results/tech.csv, 사건 : results/event.csv, 국가 : results/nation.csv)로 따로 저장.

- edge는 result/relation.csv에 저장.

In [2]:
import csv

company = [["id", "name", "description"]]
people = [["id", "name", "description"]]
tech = [["id", "name", "description"]]
event = [["id", "name", "description"]]
nation = [["id", "name", "description"]]

node_id = 0

for node_name in nodes:
    if nodes[node_name]['type'] == "기업":
        company.append([node_id, node_name, nodes[node_name]['description']])
    elif nodes[node_name]['type'] == "인물":
        people.append([node_id, node_name, nodes[node_name]['description']])
    elif nodes[node_name]['type'] == "기술":
        tech.append([node_id, node_name, nodes[node_name]['description']])
    elif nodes[node_name]['type'] == "사건":
        event.append([node_id, node_name, nodes[node_name]['description']])
    elif nodes[node_name]['type'] == "국가":
        nation.append([node_id, node_name, nodes[node_name]['description']])
    node_id += 1

f = open("results/company.csv", "w")
writer = csv.writer(f)
writer.writerows(company)
f.close()

f = open("results/people.csv", "w")
writer = csv.writer(f)
writer.writerows(people)
f.close()

f = open("results/tech.csv", "w")
writer = csv.writer(f)
writer.writerows(tech)
f.close()

f = open("results/event.csv", "w")
writer = csv.writer(f)
writer.writerows(event)
f.close()

f = open("results/nation.csv", "w")
writer = csv.writer(f)
writer.writerows(nation)
f.close()

relation = [['id', 'source', 'dest', 'description', 'score', 'time', 'url']]

edge_id = 0

for edge in edges:
    relation.append([edge_id, edge['source'], edge['dest'], edge['description'], edge['score'], edge['time'], edge['url']])
    edge_id += 1

f = open("results/relation.csv", "w")
writer = csv.writer(f)
writer.writerows(relation)
f.close()